In [ ]:
import queue
import random
import time
import threading

In [ ]:
# Define a Backlog Item class
class BacklogItem:
    def __init__(self, id, priority, is_blocked=False):
        self.id = id
        self.priority = priority
        self.is_blocked = is_blocked
    
    def __str__(self):
        return f"ID: {self.id}, Priority: {self.priority}, Blocked: {self.is_blocked}"


In [ ]:
# Create a priority queue for the backlog
backlog = queue.PriorityQueue()


In [ ]:
# Add some sample backlog items with different priorities
def create_sample_backlog():
    for i in range(10):
        backlog.put((random.randint(1, 10), BacklogItem(i, random.randint(1, 10), random.choice([True, False]))))


In [ ]:
# Worker function
def worker(worker_id):
    while True:
        # Simulate random worker disappearance (vacation/sick leave)
        if random.random() < 0.1:  # 10% chance to disappear
            print(f"Worker {worker_id} is taking a break...")
            time.sleep(random.randint(1, 5))  # Worker disappears for a random time
            continue
        
        # Try to pick a backlog item if available
        if not backlog.empty():
            priority, item = backlog.get()
            # Check if the item is blocked
            if item.is_blocked:
                print(f"Worker {worker_id} found item {item.id} is blocked, skipping it.")
                # Reinsert the blocked item back into the queue
                backlog.put((priority, item))
                time.sleep(1)  # Worker takes a break before trying again
            else:
                print(f"Worker {worker_id} is working on item {item.id} with priority {item.priority}")
                time.sleep(random.randint(1, 3))  # Simulate time taken to work on the item
        else:
            print(f"Worker {worker_id} finds the backlog empty. Taking a break...")
            time.sleep(2)


In [ ]:
# Function to simulate the scrum process
def simulate_scrum():
    create_sample_backlog()

    # Start worker threads
    num_workers = 5
    threads = []
    for i in range(num_workers):
        thread = threading.Thread(target=worker, args=(i,))
        threads.append(thread)
        thread.start()
    
    # Run for a fixed period of time
    time.sleep(20)  # Simulate for 20 seconds
    for thread in threads:
        thread.join()


In [ ]:
# Run the simulation
simulate_scrum()